2025/10/13

掌握迭代器 (__iter__, __next__) 的机制

理解生成器 (yield) 的本质与优势

体会 Python 的「协议导向设计」和「惰性求值」特性

写一个小练习：生成器版文件行统计脚本

刷 2 道数组题热身（Two Sum + Remove Duplicates）


迭代器 Iterator 可以按顺序取出元素的对象


In [ ]:
nums = [10,20,30]
it = iter(nums)

print(next(it))
print(next(it))
print(next(it))
#print(next(it))
#iter(nums)调用的nums._iter_()
#next(it)调用的it._next_()

10
20
30


StopIteration: 

In [ ]:
#自定义迭代器
class CountDown:
    def __init__(self,start):#当使用CountDown（3）的时候 start就是3创建对象并保存self.current = 3
        self.current = start #self.current相当于计数器
    def __iter__(self):#_iter_()是可迭代协议的一部分
        return self#返回自身 迭代器协议
    def __next__(self):#返回下一个元素
        if self.current<=0:
            raise StopIteration
        val = self.current
        self.current-=1
        return val
    
for x in CountDown(3):
    print(x)

3
2
1


生成器 Generator
是迭代器的特殊形式
生成器是“自动帮你写好 __iter__ 和 __next__ 的迭代器”。

In [ ]:
def countdown(n):
    while n>0:
        yield n
        n -=1

g = countdown(3)#调用不会立刻执行整个函数 要真正取值需要调用next
print(g)        # <generator object countdown at 0x...>
print(next(g))  # 3
print(next(g))  # 2
print(next(g))  # 1
#print(next(g))  # StopIteration


<generator object countdown at 0x000001A9BE123100>
3
2
1


In [11]:
# 练习 1：CountDown 倒计时迭代器

# 从 n 开始，每次迭代输出 n, n-1, …, 1，然后停止。

# 💡 思路：

# 初始化保存当前值；

# 每次 next() 输出当前值并减 1；

# 到 0 时抛 StopIteration。

class CountDown:
    def __init__(self, start):
        self.count = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.count <= 0:
            raise StopIteration
        val = self.count
        self.count-=1
        return val


for x in CountDown(5):
    print(x)



5
4
3
2
1


In [ ]:
# 练习 2：ReverseList 反向遍历迭代器

# 实现一个类，接受一个列表参数，然后 for 时从后往前输出。

# 💡 思路：

# 保存列表和当前索引；

# 从最后一个元素开始；

# 每次 __next__() 返回一个值并向前移动索引；

# 结束条件是索引 < 0。
class ReverseList:
    def __init__(self,data):
        self.data = data #保存列表
        self.index = len(data)-1 #start from the lastest
    def __iter__(self):
        return self #return the itereation self
    def __next__(self):
        if self.index< 0:#关于为什么是小于0 len(data)=3 索引 0 1 2 所以前面self.index=len(data)-1 这里是<0
            raise StopIteration
        val = self.data[self.index]#get the current value
        self.index -= 1
        return val

for x in ReverseList([10, 20, 30]):
    print(x)


30
20
10


In [20]:
# 练习 3：EvenIterator 偶数迭代器

# 迭代器接收一个列表，只输出其中的偶数。

# 💡 思路：

# 维护原列表；

# __next__() 跳过奇数；

# 没有偶数时触发 StopIteration。
class EvenIterator:
    def __init__(self, nums):
        self.data = nums
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        while self.index<len(self.data):
            val = self.data[self.index]
            self.index+=1
            if val%2 ==0:
                return val
        raise StopIteration
                
            
        

for x in EvenIterator([1, 2, 3, 4, 6]):
    print(x)


2
4
6


In [22]:
# 练习 4：FibonacciIterator 斐波那契迭代器

# 实现一个能返回前 N 个斐波那契数的迭代器。

# 💡 思路：

# 状态：a, b, count, max；

# 每次返回当前 a；

# 下一步更新 (a, b) = (b, a+b)；

# 超出次数后抛 StopIteration。


class FibonacciIterator:
    def __init__(self, n):
        self.n = n
        self.a = 0
        self.b = 1
        self.count = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.count>=self.n:
            raise StopIteration
        val = self.a
        self.a, self.b = self.b, self.a+self.b
        self.count+=1
        return val


for num in FibonacciIterator(8):
    print(num)


0
1
1
2
3
5
8
13


In [24]:
# 练习 5：FileLineIterator 文件行迭代器

# 实现一个迭代器，逐行读取文件内容（模拟 for line in open() 的行为）。

# 💡 思路：

# 在 __init__ 中打开文件；

# __next__() 每次读一行；

# 文件结尾（''）时抛 StopIteration；

# 可选：在 __del__() 里关闭文件。

class FileLineIterator:
    def __init__(self, filepath):
        self.file = open(filepath,"r",encoding="utf-8")

    def __iter__(self):
        return self

    def __next__(self):
        line = self.file.readline()
        if not line:
            self.file.close()
            raise StopIteration
        return line

for line in FileLineIterator("example.txt"):
    print(line.strip())


111111111111111
21222222222222222222
3433333333333333333333
4444444444
555555556
666666666


In [ ]:
# #关于生成管道组合 它让你能像“流水线”一样，一步步加工数据，
# 我们要做的任务是：

# 读出所有行；

# 去掉空行；

# 把所有字母转成大写；

# 打印输出。
def read_lines(path):
    with open(path,"r",encoding="utf-8") as f:
        for line in f:
            yield line.strip()#yield 可以一行一行返回文本 line.strip()可以去除换行符

def non_empty(lines):
    for line in lines:
        if line:#空字符串false 只把非空传下去

            yield line
def to_upper(lines):
    for line in lines:
        yield line.upper()

for line in to_upper(non_empty(read_lines("data.txt"))):
    print(line)

    #所以整个过程是一层层懒加载的，不会提前处理所有数据。

HELLO
PYTHON
OPENAI
